### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M11D30', 'UTC 09:40')


# Settings

In [2]:
MAXTASKS = 100
CANCELTASKS = 1

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,error_message,id,output_url,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,NaN,1511975024210,PTotWN_monthY2014M12V01,NaN,TMSOBCCOASUKYM4E4ZYGRFRX,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975024210
1,NaN,1511975022416,PTotWN_monthY2014M11V01,NaN,57RBZ7TRPV3UOIV7XSSVS2Z7,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975022416
2,NaN,1511975020608,PTotWN_monthY2014M10V01,NaN,BDIGW3FV6H5QDIGRLPGTR3HU,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975020608
3,NaN,1511975018936,PTotWN_monthY2014M09V01,NaN,JLXZPPI7SW37UQT4VMXJ56XP,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975018936
4,NaN,1511975017151,PTotWN_monthY2014M08V01,NaN,U33NVNVQTQ7BLHHO7IGTT4SD,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975017151
5,NaN,1511975015411,PTotWN_monthY2014M07V01,NaN,3PA44RBU6LAMSWETOD7ZMN24,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975015411
6,NaN,1511975013686,PTotWN_monthY2014M06V01,NaN,V6I2MWNS4VYASA3DT6RBUSGC,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975013686
7,NaN,1511975011947,PTotWN_monthY2014M05V01,NaN,ZLMOUDQX7NALOSGYJL5L5MYZ,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975011947
8,NaN,1511975010264,PTotWN_monthY2014M04V01,NaN,MDD2WUKDIE7RB7YYAS5ZCKRV,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975010264
9,NaN,1511975008590,PTotWN_monthY2014M03V01,NaN,4NSLIHVSJ4IG7OBTWUP7QW2S,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1511975008590


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)

<Task EXPORT_IMAGE: PTotWN_monthY2014M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M12V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M11V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M11V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M10V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M10V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M09V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M09V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M08V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M08V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M07V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M07V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M06V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M06V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M05V01 (READY)>
canceling <Task EXPORT_IMAGE: PTotWN_monthY2014M05V01 (READY)>
<Task EXPORT_IMAGE: PTotWN_monthY2014M04V01 (READY)>
canceling <Task EXP